# Decoding with Belief Propagation

We want to iterate the following the following equations:

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \ j } m_{\mu j}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (l)} \ \mu \tanh^{1} \left(  \hat{m}_{\mu j}  \right)     \right) + \beta(\rho_\xi)  \; \; ,  $$  

which are Eqs.(96) from the paper [Low-density parity-check codes—A statistical physics perspective](https://www.sciencedirect.com/science/article/pii/S1076567002800180), by R. Vicente, D. Saad and Y. Kabashima.

The function $ \beta(x) $ is the Nishimori temperature,

$$ \beta(x) = \frac{1}{2} \log \left(  \frac{1- \rho}{\rho}  \right) \; \; .  $$

The quantity $\rho$ is the flip probablility of the noisy channel (BSC),

$$ P ( J | J^{(0)} ) = (1 - \rho) \delta_{J, J^{(0)} } + \rho \delta_{J, -J^{(0)} }   \; \; , $$

whereas the prior distribution for each meassage bit is assumed to be

$$ P ( S_j ) = (1 - \rho_\xi) \delta_{+1, S_j }  +  \rho_\xi \delta_{-1, S_j }  \; \; .  $$

The object $ {\cal L} (\mu)$ represents the set of $K$ non-zero elements on the row $\mu$ of the code generator matrix ${\cal G}$ (the one which adds redundancy), 

$$  {\cal L} (\mu) = \langle i_1, i_2, ..., i_K \rangle  \; \; .  $$

The are $C$ non-zero elements per column on the matrix ${\cal G}$:

$$ \sum_{\mu : j \in {\cal L}(\mu)} i_j = C \; \; ; \; \; \forall j = 1, ..., K \; \; . $$

The object $ {\cal M} (j)$ represents the set of all index sets that contain $j$.